# 【概要】

- モデル：Single LightGBM
- 特徴量：n個　（メモリが足りないから、全て別のノートブックで作成し、inputから使う）
- Local CV：
- Public LB：

# 読み込み

In [1]:
%%time

from google.cloud import storage as gcs
import pandas as pd
from io import BytesIO

project_id = "Project kaggle"
bucket_name = "kaggle_riiid_ynum"
gcs_path = "input/train.csv"

client = gcs.Client(project_id)
bucket = client.get_bucket(bucket_name)

# GCSにあるダウンロードしたいファイルを指定
blob = bucket.blob(gcs_path)

# DataFrameとして受け取る(ヘッダーありのCSVを想定)
train_df = pd.read_csv(BytesIO(blob.download_as_string()))

# 型の変換
# data_types_dict = {
#     'timestamp': 'int64',
#     'user_id': 'int32', 
#     'content_id': 'int16', 
#     'content_type_id':'int8', 
#     'task_container_id': 'int16',
#     'user_answer': 'int8',
#     'answered_correctly': 'int8', 
#     'prior_question_elapsed_time': 'float32', 
#     'prior_question_had_explanation': 'bool'
# }

# train_df = train_df.astype(data_types_dict)

CPU times: user 1min 55s, sys: 23.7 s, total: 2min 19s
Wall time: 2min 20s


In [2]:
# 型の変換
data_types_dict = {
    'timestamp': 'int64',
    'user_id': 'int32', 
    'content_id': 'int16', 
    'content_type_id':'int8', 
    'task_container_id': 'int16',
    'user_answer': 'int8',
    'answered_correctly': 'int8', 
    'prior_question_elapsed_time': 'float32', 
    'prior_question_had_explanation': 'bool'
}

train_df = train_df.astype(data_types_dict)

In [3]:
# directory
# %cd /Users/aoi/Documents/kaggle/riiid/data

In [4]:
#gpuを使用するとき
#!nvidia-smi

In [5]:
# !pip install ../input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl > /dev/null 2>&1

In [6]:
import numpy as np
import pandas as pd
import psutil
import joblib
import os
import time
import datetime
from collections import defaultdict
# import eli5
# from eli5.sklearn import PermutationImportance
# import datatable as dt
import lightgbm as lgb
from matplotlib import pyplot as plt
import seaborn as sns
# import riiideducation
import random
from sklearn.metrics import roc_auc_score
import sklearn.metrics as metrics
import gc
from tqdm.notebook import tqdm
from sklearn.preprocessing import StandardScaler
import pickle
#models
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from catboost import Pool
from sklearn.linear_model import RidgeCV, RidgeClassifierCV
#HDKIM SAKT
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

_ = np.seterr(divide='ignore', invalid='ignore')

#列を…で省略させずに、全部表示(1000列上限にしてる)させるための設定
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

def seed_everything(seed=20):
        random.seed(seed)
        np.random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed_everything(seed=20)

In [7]:
%%time

#各列のデータタイプを辞書型で指定し、data_types_dictに保存

# data_types_dict = {
#     'timestamp': 'int64',
#     'user_id': 'int32', 
#     'content_id': 'int16', 
#     'content_type_id':'int8', 
#     'task_container_id': 'int16',
#     'user_answer': 'int8',
#     'answered_correctly': 'int8', 
#     'prior_question_elapsed_time': 'float32', 
#     'prior_question_had_explanation': 'bool'
# }

target = 'answered_correctly'

# train_df = dt.fread('../input/riiid-test-answer-prediction/train.csv', columns=set(data_types_dict.keys())).to_pandas()
# train_df = pd.read_pickle('./input/train.pkl')

train_df_use_cols = [
#     'row_id',
    'timestamp',
    'user_id',
    'content_id',
#     'content_type_id',
    'task_container_id',
#     'user_answer',
    'answered_correctly',
    'prior_question_elapsed_time',
    'prior_question_had_explanation'
]

train_df = train_df[train_df_use_cols]

CPU times: user 1.12 s, sys: 20.3 ms, total: 1.14 s
Wall time: 1.13 s


In [8]:
train_df = train_df[train_df[target] != -1].reset_index(drop=True)

# trainに特徴量を結合

ただ、作成した特徴量は

①train_dfと同じ行数ある特徴量（cum系）

②なんかのidをキーとしてtrain_dfとleft joinする特徴量

があり、ここでは①を結合。②はメモリの事情でtrainとvalidに分割した後でそれぞれに結合させる。

trainは約1億行あって重いから、ランダムに「train_mini_len行」抽出しtrain_miniにいれる

In [9]:
train_mini_len = 2*10*1000*1000
# train_mini_len = len(train_df)

In [10]:
train_mini = train_df.sample(n=train_mini_len, random_state=42)
# train_mini = train_df.copy()

del train_df
gc.collect()

66

In [11]:
train_mini.shape

(20000000, 7)

In [12]:
path = 'features/fe_user_ans.csv'
blob = bucket.blob(path)
feature = pd.read_csv(BytesIO(blob.download_as_string()))

# feature = feature[['user_cnt', 'user_corcnt', 'user_inccnt', 'user_corrate', 'user_diff_sum_corrate', 'user_diff_avg_corrate']]

feature_mini = feature.sample(n=train_mini_len, random_state=42)
del feature

train_mini = pd.concat([train_mini, feature_mini], axis=1)
del feature_mini
gc.collect()

5

In [13]:
path = 'features/fe_user_task.csv'
blob = bucket.blob(path)
feature = pd.read_csv(BytesIO(blob.download_as_string()))

feature = feature[['task_container_id_back']]

feature_mini = feature.sample(n=train_mini_len, random_state=42)
del feature

train_mini = pd.concat([train_mini, feature_mini], axis=1)
del feature_mini
gc.collect()

5

In [14]:
path = 'features/fe_user_roll.csv'
blob = bucket.blob(path)
feature = pd.read_csv(BytesIO(blob.download_as_string()))

feature = feature[['user_roll10_corrate', 'user_roll30_corrate']]

feature_mini = feature.sample(n=train_mini_len, random_state=42)
del feature

train_mini = pd.concat([train_mini, feature_mini], axis=1)
del feature_mini
gc.collect()

5

In [15]:
path = 'features/fe_user_time.csv'
blob = bucket.blob(path)
feature = pd.read_csv(BytesIO(blob.download_as_string()))

feature = feature[['user_diff_sum_time', 'user_diff_avg_time']]

feature_mini = feature.sample(n=train_mini_len, random_state=42)
del feature

train_mini = pd.concat([train_mini, feature_mini], axis=1)
del feature_mini
gc.collect()

5

In [16]:
path = 'features/fe_user_past.csv'
blob = bucket.blob(path)
feature = pd.read_csv(BytesIO(blob.download_as_string()))

feature = feature[['user_past_content_cnt']]

feature_mini = feature.sample(n=train_mini_len, random_state=42)
del feature

train_mini = pd.concat([train_mini, feature_mini], axis=1)
del feature_mini
gc.collect()

5

In [17]:
path = 'features/fe_user_part_ans.csv'
blob = bucket.blob(path)
feature = pd.read_csv(BytesIO(blob.download_as_string()))

feature_mini = feature.sample(n=train_mini_len, random_state=42)
del feature

train_mini = pd.concat([train_mini, feature_mini], axis=1)
del feature_mini
gc.collect()

5

In [18]:
path = 'features/fe_user_part_diff_corrate.csv'
blob = bucket.blob(path)
feature = pd.read_csv(BytesIO(blob.download_as_string()))

feature_mini = feature.sample(n=train_mini_len, random_state=42)
del feature

train_mini = pd.concat([train_mini, feature_mini], axis=1)
del feature_mini
gc.collect()

5

In [19]:
path = 'features/fe_user_part_roll.csv'
blob = bucket.blob(path)
feature = pd.read_csv(BytesIO(blob.download_as_string()))

feature = feature[['user_part_roll5_corrate', 'user_part_roll15_corrate']]

feature_mini = feature.sample(n=train_mini_len, random_state=42)
del feature

train_mini = pd.concat([train_mini, feature_mini], axis=1)
del feature_mini
gc.collect()

5

In [20]:
path = 'features/fe_user_w2v.csv'
blob = bucket.blob(path)
feature = pd.read_csv(BytesIO(blob.download_as_string()))

feature = feature[['user_w2v_corrate']]

feature_mini = feature.sample(n=train_mini_len, random_state=42)
del feature

train_mini = pd.concat([train_mini, feature_mini], axis=1)
del feature_mini
gc.collect()

5

In [21]:
path = 'features/fe_user_tags1_ans.csv'
blob = bucket.blob(path)
feature = pd.read_csv(BytesIO(blob.download_as_string()))

feature = feature[['user_tags1_corcnt', 'user_tags1_inccnt' ,'user_tags1_corrate']]

feature_mini = feature.sample(n=train_mini_len, random_state=42)
del feature

train_mini = pd.concat([train_mini, feature_mini], axis=1)
del feature_mini
gc.collect()

5

In [22]:
path = 'features/fe_013_cum.csv'
blob = bucket.blob(path)
feature = pd.read_csv(BytesIO(blob.download_as_string()))

feature_mini = feature.sample(n=train_mini_len, random_state=42)
del feature

train_mini = pd.concat([train_mini, feature_mini], axis=1)
del feature_mini
gc.collect()

5

In [23]:
path = 'features/fe_014_cum.csv'
blob = bucket.blob(path)
feature = pd.read_csv(BytesIO(blob.download_as_string()))

feature_mini = feature.sample(n=train_mini_len, random_state=42)
del feature

train_mini = pd.concat([train_mini, feature_mini], axis=1)
del feature_mini
gc.collect()

5

In [24]:
path = 'features/fe_015_cum.csv'
blob = bucket.blob(path)
feature = pd.read_csv(BytesIO(blob.download_as_string()))

feature_mini = feature.sample(n=train_mini_len, random_state=42)
del feature

train_mini = pd.concat([train_mini, feature_mini], axis=1)
del feature_mini
gc.collect()

5

別のノートブックで作成した特徴量同士を結合（concat）して１つのデータフレーム（fe）を作成

In [25]:
# %%time
# # 特徴量のpickleファイルがあるパスを指定
# fe_user_ans = 'features/fe_user_ans.csv'
# fe_user_task = 'features/fe_user_task.csv'
# fe_user_roll = 'features/fe_user_roll.csv'
# fe_user_time = 'features/fe_user_time.csv'
# fe_user_past = 'features/fe_user_past.csv'
# fe_user_part_ans = 'features/fe_user_part_ans.csv'
# fe_user_part_diff_corrate = 'features/fe_user_part_diff_corrate.csv'
# fe_user_part_roll = 'features/fe_user_part_roll.csv'
# fe_user_tags1_ans = 'features/fe_user_tags1_ans.csv'
# fe_user_w2v = 'features/fe_user_w2v.csv'
# fe_013_cum = 'features/fe_013_cum.csv'
# fe_014_cum = 'features/fe_014_cum.csv'
# fe_015_cum = 'features/fe_015_cum.csv'

# path_list = [
#     fe_user_ans,
#     fe_user_task,
#     fe_user_roll,
#     fe_user_time,
#     fe_user_past,
#     fe_user_part_ans,
#     fe_user_part_diff_corrate,
#     fe_user_part_roll,
#     fe_user_tags1_ans,
#     fe_user_w2v,
#     fe_013_cum,
#     fe_014_cum,
#     fe_015_cum
# ]

# #特徴量だけのデータフレームを作成
# fe = pd.DataFrame()

# for path in path_list:
    
#     # GCSにあるダウンロードしたいファイルを指定
#     blob = bucket.blob(path)

#     # DataFrameとして受け取る(ヘッダーありのCSVを想定)
#     feature = pd.read_csv(BytesIO(blob.download_as_string()))
# #     feature = pd.read_pickle(path)

#     feature_mini = feature.sample(n=train_mini_len, random_state=42)

# #     fe = pd.concat([fe, feature], axis=1)

#     del feature
#     gc.collect()
    
#     fe = pd.concat([fe, feature_mini], axis=1)
    
#     del feature_mini
#     gc.collect()

train_miniとfeの結合（concat）

In [26]:
# train_mini = pd.concat([train_mini, fe], axis=1)

# del fe
# gc.collect()

In [27]:
train_mini.shape

(20000000, 38)

# train_miniをtrainとvalidに分割

train_miniの中からvalidの新規ユーザーとして使うユーザーをランダムに選ぶ

In [28]:
users = train_mini['user_id'].drop_duplicates()
users = users.sample(frac=0.025, random_state=42)

users_df = pd.DataFrame()
users_df['user_id'] = users.values

valid_df_newuser = pd.merge(train_mini, users_df, on=['user_id'], how='inner',right_index=True)

del users_df
del users
gc.collect()

train_mini.drop(valid_df_newuser.index, inplace=True)
gc.collect()

0

train_miniとvalid_df_newuserそれぞれに②の特徴量を結合

In [29]:
#特徴量のpickleファイルがあるパスを指定
path = 'features/fe_content.csv'
blob = bucket.blob(path)
features = pd.read_csv(BytesIO(blob.download_as_string()))

In [30]:
features = features[[
    'bundle_id',
    'part',
    'content_cnt',
#         'content_corcnt',
    'content_inccnt',
    'content_corrate',
#         'content_corstd',
#         'content_sum_time',
    'content_avg_time',
#         'content_expl_sum',
    'content_expl_avg',
#         'content_lecture_cnt',
#         'content_lecture_sum_time',
#         'content_lecture_avg_time',

#         'new_content_cnt',
    'new_content_corcnt',
#         'new_content_inccnt',
    'new_content_corrate',

#         'bundle_cnt',
    'same_bundle_cnt',
#         'bundle_unicnt',
#         'bundle_corcnt',
#         'bundle_inccnt',
    'bundle_corrate',
#         'bundle_corstd',

#         'part_cnt',
#         'part_corcnt',
#         'part_inccnt',
#         'part_corrate',
#         'part_corstd',

#         'cluster',
#         'cluster_cnt',
#         'cluster_corcnt',
#         'cluster_inccnt',
#         'cluster_corrate',
#         'cluster_corstd',

    'tags1',
#         'tags1_cnt',
#         'tags1_corcnt',
#         'tags1_inccnt',
#         'tags1_corrate',
#         'tags1_corstd',
    'tags2',
#         'tags3',
#         'tags4',
#         'tags5',
#         'tags6',

#         'le_tags',
    'new_w2v',
#     'w2v',

#         'question_had_explanation_mean',
#         'tags_lsi',
#         'tag_acc_count',
#         'tag_acc_max',
#         'tag_acc_min',  
]]
#     feature = feature.drop(columns='content_id')

In [31]:
train_mini = pd.concat([train_mini.reset_index(drop=True), features.reindex(train_mini['content_id'].values).reset_index(drop=True)], axis=1)
valid_df_newuser = pd.concat([valid_df_newuser.reset_index(drop=True), features.reindex(valid_df_newuser['content_id'].values).reset_index(drop=True)], axis=1)
    
# del feature
gc.collect()

5

In [36]:
train_mini.shape

(17755881, 52)

valid_dfとvalid_df_newuserを結合してvalid完成

In [35]:
valid_df = train_mini.sample(frac=0.09, random_state=42)
train_mini.drop(valid_df.index, inplace=True)

valid_df = valid_df.append(valid_df_newuser)

del valid_df_newuser
gc.collect()

0

trainとvalidが何行何列か確認

In [37]:
print('trainの行数と列数：', train_mini.shape)
print('validの行数と列数：', valid_df.shape)

trainの行数と列数： (17755881, 52)
validの行数と列数： (2244119, 52)


特徴量に使う列名をfeaturesに入れる

In [38]:
features = [
    
### デフォルトで存在する特徴量 ###
#     'user_id',
    'content_id',
    'timestamp',
    'task_container_id',
    'prior_question_elapsed_time',
    'prior_question_had_explanation',
    'bundle_id',
    'part',    
    
### 問題に関する特徴量 ###
    
    'content_cnt',
#     'content_corcnt',
    'content_inccnt',
    'content_corrate',
#     'content_corstd',
#     'content_sum_time',
    'content_avg_time',
#     'content_expl_sum',
    'content_expl_avg',
#     'content_lecture_cnt',
#     'content_lecture_sum_time',
#     'content_lecture_avg_time',
    
#     'new_content_cnt',
    'new_content_corcnt',
#     'new_content_inccnt',
    'new_content_corrate',
    
#     'bundle_cnt',
    'same_bundle_cnt',
#     'bundle_unicnt',
#     'bundle_corcnt',
#     'bundle_inccnt',
    'bundle_corrate',
#     'bundle_corstd',

#     'part_cnt',
#     'part_corcnt',
#     'part_inccnt',
#     'part_corrate',
#     'part_corstd',
    
#     'cluster',
#     'cluster_cnt',
#     'cluster_corcnt',
#     'cluster_inccnt',
#     'cluster_corrate',
#     'cluster_corstd',
    
    'tags1',
#     'tags1_cnt',
#     'tags1_corcnt',
#     'tags1_inccnt',
#     'tags1_corrate',
#     'tags1_corstd',
    'tags2',
#     'tags3',
#     'tags4',
#     'tags5',
#     'tags6',
    
#     'le_tags',
    'new_w2v',
#     'w2v',
    
#     'question_had_explanation_mean',
#     'tags_lsi',
#     'tag_acc_count',
#     'tag_acc_max',
#     'tag_acc_min',
    
### ユーザーに関する特徴量 ###
    
#     'timestamp_rank',
#     'timestamp_diff',
    
#     'frequency',
    'elapsed_time_u_avg',
    'explanation_u_avg',
    'timestamp_u_correct_recency',
    'timestamp_u_incorrect_recency',
    'timestamp_u_diff_0_1',
    'timestamp_u_diff_1_2',
    'timestamp_u_diff_2_3',
    
#     'task_container_id_max',
    'task_container_id_back',
    
    'user_cnt',
    'user_corcnt',
    'user_inccnt',
    'user_corrate',
    
    'user_diff_sum_corrate',
    'user_diff_avg_corrate',
    
#     'user_roll3_corrate',
#     'user_roll5_corrate',
    'user_roll10_corrate',
#     'user_roll20_corrate',
    'user_roll30_corrate',
#     'user_roll50_corrate',
    
#     'user_sum_time',
#     'user_avg_time',
    'user_diff_sum_time',
    'user_diff_avg_time',
    
#     'user_expl_avg',
    
#     'diagnostic_test',
#     'diagnostic_test_2',
    
    'user_past_content_cnt',
#     'user_past_expl_cnt',
#     'user_past_lecture_cnt',
    
    'user_part_cnt',
    'user_part_corcnt', 
    'user_part_inccnt',
    'user_part_corrate',
    
    'user_part_diff_sum_corrate',
    'user_part_diff_avg_corrate',
    
#     'user_part_roll3_corrate',
    'user_part_roll5_corrate',
#     'user_part_roll10_corrate',
    'user_part_roll15_corrate',
#     'user_part_roll20_corrate',
#     'user_part_roll30_corrate',
    
#     'user_w2v_cnt',
#     'user_w2v_corcnt',
#     'user_w2v_inccnt',
    'user_w2v_corrate',
    
#     'user_tags1_cnt',
    'user_tags1_corcnt',
    'user_tags1_inccnt',
    'user_tags1_corrate'
]

# カテゴリ変数
categorical_columns = [
    
### デフォルトで存在する特徴量 ###
#     'user_id',
    'content_id',
#     'timestamp',
#     'task_container_id',
    'prior_question_had_explanation',
    'bundle_id',
    'part',    
    
### 問題に関する特徴量 ###
    
    'same_bundle_cnt',
    
#     'cluster',

    'tags1',
    'tags2',
#     'tags3',
#     'tags4',
#     'tags5',
#     'tags6',
    'new_w2v',
#     'w2v',
#     'tags_lsi',
    
### ユーザーに関する特徴量 ###
    
#     'timestamp_rank',
    
#     'task_container_id_max',
    'task_container_id_back',
    
#     'diagnostic_test',
#     'diagnostic_test_2',
]

In [39]:
print('特徴量の個数：', len(features), '個')

特徴量の個数： 50 個


In [40]:
# %%capture
# '''
#エラー起こらないように型を変える
train_mini['prior_question_had_explanation'].fillna(False, inplace=True)
valid_df['prior_question_had_explanation'].fillna(False, inplace=True)

train_mini['prior_question_had_explanation'] = train_mini['prior_question_had_explanation'].astype(bool)
valid_df['prior_question_had_explanation'] = valid_df['prior_question_had_explanation'].astype(bool)

train_mini['prior_question_had_explanation'] = train_mini['prior_question_had_explanation'].astype(int)
valid_df['prior_question_had_explanation'] = valid_df['prior_question_had_explanation'].astype(int)
# '''

### features確認

In [41]:
%%capture
# データが巨大で時間かかるので省略
'''
for col in features:
    try:
        plt.title(col)
        sns.distplot(train_mini[col].fillna(0))#.sample(1000000, random_state=42)
        plt.show()
    except:
        print('='*30)
        print(f'ERROR IN {col}')
        print('='*30)
'''

### 特徴量を正規分布に近い形に変形する(対数変換など)
- スコアが上がることがある

In [42]:
# # 右裾が長い特徴量（対数変換）
# log_features = [
#     'timestamp',
#     'task_container_id',
#     'prior_question_elapsed_time',
#     'content_cnt',
#     'content_corcnt',
#     'content_sum_time',
#     'content_avg_time',
#     'content_expl_sum',
#     'content_lecture_cnt',
#     'content_lecture_sum_time',
#     'content_lecture_avg_time',
#     'new_content_cnt',
#     'new_content_corcnt',
#     'new_content_inccnt',
#     'bundle_cnt',
#     'bundle_unicnt',
#     'bundle_corcnt',
#     'bundle_inccnt',
#     'timestamp_rank',
#     'timestamp_diff',
#     'task_container_id_max',
#     'user_cnt',
#     'user_corcnt',
#     'user_inccnt',
#     'user_sum_time',
#     'user_avg_time',
#     'user_part_cnt',
#     'user_part_corcnt',
#     'user_part_inccnt',
#     'user_tags1_cnt',
#     'user_tags1_corcnt',
#     'user_tags1_inccnt'
# ]

# # 右に裾が長い特徴量（ルート変換）
# sqrt_features = [
    
# ]

# # 左に裾が長い特徴量（未解決）
# left_long_features = [
#     'content_corstd',
#     'content_expl_avg',
#     'bundle_corstd'
# ]

# # 両側に裾が長い特徴量（絶対値対数変換）
# abs_log_features = [

# ]

# # 両側に裾が長い特徴量（絶対値ルート変換）
# abs_sqrt_features = [
#     'user_diff_sum_corrate',
#     'user_diff_sum_time',
#     'user_part_diff_sum_corrate'
# ]

# # 外れ値を置き換える（clipping）特徴量
# clip_features = [
#     'user_diff_avg_time'
# ]

In [43]:
# # 対数変換
# for col in log_features:
#     train_mini[col] = np.log1p(train_mini[col])
#     valid_df[col] = np.log1p(valid_df[col])

# # ルート変換
# for col in sqrt_features:
#     train_mini[col] = np.sqrt(train_mini[col])
#     valid_df[col] = np.sqrt(valid_df[col])

# # 絶対値対数変換
# for col in abs_log_features:
#     train_mini[col] = np.sign(train_mini[col])*np.log(np.abs(train_mini[col]))
#     valid_df[col] = np.sign(valid_df[col])*np.log(np.abs(valid_df[col]))

# # 絶対値ルート変換
# for col in abs_sqrt_features:
#     train_mini[col] = np.sign(train_mini[col])*np.sqrt(np.abs(train_mini[col]))
#     valid_df[col] = np.sign(valid_df[col])*np.sqrt(np.abs(valid_df[col]))

# # 外れ値を置き換える（clipping）
# for col in abs_sqrt_features:

#     col_all = train_mini[col] # + valid_df[col]
                
#     p01 = col_all.quantile(0.01)
#     p99 = col_all.quantile(0.99)

#     train_mini[col] = train_mini[col].clip(p01, p99)                                            
#     valid_df[col] = valid_df[col].clip(p01, p99)

#     del col_all   

# gc.collect()                           

### features再度確認

In [44]:
%%capture
'''
for col in features:
    try:
        plt.title(col)
        sns.distplot(train_mini[col].fillna(0))#.sample(1000000, random_state=42)
        plt.show()
    except:
        print('='*30)
        print(f'ERROR IN {col}')
        print('='*30)
'''

### わかりやすくする

In [45]:
#わかりやすく置き換えているだけ
tr = train_mini
del train_mini
val = valid_df
del valid_df

gc.collect()

feature_cols = features
cat_feature_cols = categorical_columns
num_feature_cols = list(set(feature_cols) - set(cat_feature_cols))

target_col = target

In [46]:
tr = tr[tr['answered_correctly'].notnull()]
val = val[val['answered_correctly'].notnull()]

## モデル定義

### XGB

In [47]:
# def train_xgb(train, valid, feature_cols, target, num_iter=10000):

#     xgb_params = {
#         'objective':'binary:logistic',
#         'eval_metric':'auc',
#         'eta':0.01,
#         'seed':42,
#         'max_depth':10,
#         'tree_method': 'gpu_hist' if torch.cuda.is_available() else 'auto',
#     }

#     train_set = xgb.DMatrix(train[feature_cols], train[target])
#     valid_set = xgb.DMatrix(valid[feature_cols], valid[target])

#     xgb_model = xgb.train(xgb_params, train_set, 
#                     evals = [(train_set, 'train'),(valid_set, 'valid')],
#                     num_boost_round = num_iter,
#                     early_stopping_rounds=10,
#                     verbose_eval=100)
    
#     pred = xgb_model.predict(xgb.DMatrix(valid[feature_cols]))
#     valid_auc = metrics.roc_auc_score(valid[target_col], pred)

#     return valid_auc, xgb_model

### LGBM

In [48]:
# def train_lgbm(train, valid, feature_cols, categorical_feature_cols, target, num_iter=10000):
    
#     lgb_params = {
#     'num_leaves': 350,
#     'max_bin':700,
#     'min_child_weight': 0.03454472573214212,
#     'feature_fraction': 0.58,
#     'bagging_fraction': 0.58,
#     #'min_data_in_leaf': 106,
#     'objective': 'binary',
#     'max_depth': -1,
#     'learning_rate': 0.01,
#     "boosting_type": "gbdt",
#     "bagging_seed": 11,
#     "metric": 'auc',
#     "verbosity": -1,
#     'reg_alpha': 0.3899927210061127,
#     'reg_lambda': 0.6485237330340494,
#     'random_state': 47
#     }
    
#     train_set = lgb.Dataset(train[feature_cols], train[target])
#     valid_set = lgb.Dataset(valid[feature_cols], valid[target])

#     lgb_model = lgb.train(lgb_params, train_set,
#                     categorical_feature=categorical_feature_cols,
#                     valid_sets=[train_set, valid_set],
#                     valid_names = ['train','valid'],
#                     num_boost_round = num_iter,
#                     early_stopping_rounds=10,
#                     verbose_eval=100)

#     #pred = lgb_model.predict(train[feature_cols])
#     #train_loss = metrics.roc_auc_score(train[target_col], pred)
#     pred = lgb_model.predict(valid[feature_cols])
#     valid_loss = metrics.roc_auc_score(valid[target], pred)
#     #重要度
#     feature_imp = pd.DataFrame(sorted(zip(lgb_model.feature_importance(), feature_cols)), columns=['Value','Feature'])
#     plt.figure(figsize=(20, 10))
#     sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
#     plt.show()
    
#     return valid_loss, lgb_model


### CatBoost

In [49]:
def train_ctb(train, valid, feature_cols, target, num_iter=100000):
    
    ctb_params = {
        'loss_function': 'Logloss',
        'eval_metric': 'AUC',
        'task_type': 'GPU' if torch.cuda.is_available() else 'CPU',
#        'grow_policy': 'Lossguide',
        'iterations': num_iter,
        'learning_rate': 0.01,
        'random_seed': 42,
#        'l2_leaf_reg': 1e-1,
        'depth': 9,
#        'max_leaves': 10,
#        'border_count': 128,
        'verbose': 100,
        'early_stopping_rounds': 10,
    }
    
    train_set = Pool(train[feature_cols], train[target])
    valid_set = Pool(valid[feature_cols], valid[target])
    
    ctb_model = ctb.CatBoost(ctb_params)
    ctb_model.fit(train_set, eval_set=[valid_set], use_best_model=True)
    #pred = ctb_model.predict(valid_set, prediction_type='Class')
    valid_auc = ctb_model.get_best_score()['validation']['AUC']
    
    return valid_auc, ctb_model

### SAKT

In [50]:
# %%time
# target = 'answered_correctly'

# # train_df = dt.fread('../input/riiid-test-answer-prediction/train.csv', columns=set(data_types_dict.keys())).to_pandas()

# train_df = pd.read_pickle('./input/train.pkl')

# train_df_use_cols = [
# #     'row_id',
#     'timestamp',
#     'user_id',
#     'content_id',
#     'content_type_id',
#     'task_container_id',
# #     'user_answer',
#     'answered_correctly',
# #     'prior_question_elapsed_time',
# #     'prior_question_had_explanation'
# ]

# train_df = train_df[train_df_use_cols]

# train_df = train_df[train_df[target] != -1].reset_index(drop=True)
# train_df = train_df[train_df.content_type_id == False]
# train_df = train_df.sort_values(['timestamp'], ascending=True)
# train_df.reset_index(drop=True, inplace=True)

In [51]:
# TRAIN_SAMPLES = 320000
# MAX_SEQ = 150
# MIN_SAMPLES = 5
# EMBED_DIM = 128
# DROPOUT_RATE = 0.2
# LEARNING_RATE = 1e-3
# MAX_LEARNING_RATE = 1e-2
# EPOCHS = 50
# TRAIN_BATCH_SIZE = 256

In [52]:
# skills = train_df["content_id"].unique()
# #joblib.dump(skills, "skills.pkl.zip")
# n_skill = len(skills)
# print("number skills", len(skills))

In [53]:
# group = train_df[['user_id', 'content_id', 'answered_correctly']].groupby('user_id').apply(lambda r: (
#             r['content_id'].values,
#             r['answered_correctly'].values))

# joblib.dump(group, "group.pkl.zip")
# del train_df
# gc.collect()

In [54]:
# class SAKTDataset(Dataset):
#     def __init__(self, group, n_skill, min_samples=1, max_seq=128):
#         super(SAKTDataset, self).__init__()
#         self.max_seq = max_seq
#         self.n_skill = n_skill
#         self.samples = {}
        
#         self.user_ids = []
#         for user_id in group.index:
#             q, qa = group[user_id]
#             if len(q) < min_samples:
#                 continue
            
#             # Main Contribution
#             if len(q) > self.max_seq:
#                 total_questions = len(q)
#                 initial = total_questions % self.max_seq
#                 if initial >= min_samples:
#                     self.user_ids.append(f"{user_id}_0")
#                     self.samples[f"{user_id}_0"] = (q[:initial], qa[:initial])
#                 for seq in range(total_questions // self.max_seq):
#                     self.user_ids.append(f"{user_id}_{seq+1}")
#                     start = initial + seq * self.max_seq
#                     end = start + self.max_seq
#                     self.samples[f"{user_id}_{seq+1}"] = (q[start:end], qa[start:end])
#             else:
#                 user_id = str(user_id)
#                 self.user_ids.append(user_id)
#                 self.samples[user_id] = (q, qa)
    
#     def __len__(self):
#         return len(self.user_ids)

#     def __getitem__(self, index):
#         user_id = self.user_ids[index]
#         q_, qa_ = self.samples[user_id]
#         seq_len = len(q_)

#         q = np.zeros(self.max_seq, dtype=int)
#         qa = np.zeros(self.max_seq, dtype=int)
#         if seq_len == self.max_seq:
#             q[:] = q_
#             qa[:] = qa_
#         else:
#             q[-seq_len:] = q_
#             qa[-seq_len:] = qa_
        
#         target_id = q[1:]
#         label = qa[1:]

#         x = np.zeros(self.max_seq-1, dtype=int)
#         x = q[:-1].copy()
#         x += (qa[:-1] == 1) * self.n_skill

#         return x, target_id, label

In [55]:
# train_indexes = list(group.index)[:TRAIN_SAMPLES]
# valid_indexes = list(group.index)[TRAIN_SAMPLES:]
# train_group = group[group.index.isin(train_indexes)]
# valid_group = group[group.index.isin(valid_indexes)]
# del group, train_indexes, valid_indexes
# print(len(train_group), len(valid_group))

In [56]:
# train_dataset = SAKTDataset(train_group, n_skill, min_samples=MIN_SAMPLES, max_seq=MAX_SEQ)
# train_dataloader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=8)
# valid_dataset = SAKTDataset(valid_group, n_skill, max_seq=MAX_SEQ)
# valid_dataloader = DataLoader(valid_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=False, num_workers=8)

In [57]:
# class FFN(nn.Module):
#     def __init__(self, state_size=200):
#         super(FFN, self).__init__()
#         self.state_size = state_size
        
#         self.lr1 = nn.Linear(state_size, state_size)
#         self.relu = nn.ReLU()
#         self.lr2 = nn.Linear(state_size, state_size)
#         self.dropout = nn.Dropout(0.2)
    
#     def forward(self, x):
#         x = self.lr1(x)
#         x = self.relu(x)
#         x = self.lr2(x)
#         return self.dropout(x)

# def future_mask(seq_length):
#     future_mask = np.triu(np.ones((seq_length, seq_length)), k=1).astype('bool')
#     return torch.from_numpy(future_mask)


# class SAKTModel(nn.Module):
#     def __init__(self, n_skill, max_seq=128, embed_dim=128, dropout_rate=0.2):
#         super(SAKTModel, self).__init__()
#         self.n_skill = n_skill
#         self.embed_dim = embed_dim

#         self.embedding = nn.Embedding(2*n_skill+1, embed_dim)
#         self.pos_embedding = nn.Embedding(max_seq-1, embed_dim)
#         self.e_embedding = nn.Embedding(n_skill+1, embed_dim)

#         self.multi_att = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=8, dropout=dropout_rate)

#         self.dropout = nn.Dropout(dropout_rate)
#         self.layer_normal = nn.LayerNorm(embed_dim) 

#         self.ffn = FFN(embed_dim)
#         self.pred = nn.Linear(embed_dim, 1)
    
#     def forward(self, x, question_ids):
#         device = x.device        
#         x = self.embedding(x)
#         pos_id = torch.arange(x.size(1)).unsqueeze(0).to(device)

#         pos_x = self.pos_embedding(pos_id)
#         x = x + pos_x

#         e = self.e_embedding(question_ids)

#         x = x.permute(1, 0, 2) # x: [bs, s_len, embed] => [s_len, bs, embed]
#         e = e.permute(1, 0, 2)
#         att_mask = future_mask(x.size(0)).to(device)
#         att_output, att_weight = self.multi_att(e, x, x, attn_mask=att_mask)
#         att_output = self.layer_normal(att_output + e)
#         att_output = att_output.permute(1, 0, 2) # att_output: [s_len, bs, embed] => [bs, s_len, embed]

#         x = self.ffn(att_output)
#         x = self.layer_normal(x + att_output)
#         x = self.pred(x)

#         return x.squeeze(-1), att_weight

In [58]:
# def train_fn(model, dataloader, optimizer, scheduler, criterion, device="cpu"):
#     model.train()

#     train_loss = []
#     num_corrects = 0
#     num_total = 0
#     labels = []
#     outs = []

#     for item in dataloader:
#         x = item[0].to(device).long()
#         target_id = item[1].to(device).long()
#         label = item[2].to(device).float()
#         target_mask = (target_id != 0)

#         optimizer.zero_grad()
#         output, _, = model(x, target_id)
#         loss = criterion(output, label)
#         loss.backward()
#         optimizer.step()
#         scheduler.step()
#         train_loss.append(loss.item())

#         output = torch.masked_select(output, target_mask)
#         label = torch.masked_select(label, target_mask)
#         pred = (torch.sigmoid(output) >= 0.5).long()
        
#         num_corrects += (pred == label).sum().item()
#         num_total += len(label)

#         labels.extend(label.view(-1).data.cpu().numpy())
#         outs.extend(output.view(-1).data.cpu().numpy())

#     acc = num_corrects / num_total
#     auc = roc_auc_score(labels, outs)
#     loss = np.mean(train_loss)

#     return loss, acc, auc

In [59]:
# def valid_fn(model, dataloader, criterion, device="cpu"):
#     model.eval()

#     valid_loss = []
#     num_corrects = 0
#     num_total = 0
#     labels = []
#     outs = []

#     for item in dataloader:
#         x = item[0].to(device).long()
#         target_id = item[1].to(device).long()
#         label = item[2].to(device).float()
#         target_mask = (target_id != 0)

#         output, _, = model(x, target_id)
#         loss = criterion(output, label)
#         valid_loss.append(loss.item())

#         output = torch.masked_select(output, target_mask)
#         label = torch.masked_select(label, target_mask)
#         pred = (torch.sigmoid(output) >= 0.5).long()
        
#         num_corrects += (pred == label).sum().item()
#         num_total += len(label)

#         labels.extend(label.view(-1).data.cpu().numpy())
#         outs.extend(output.view(-1).data.cpu().numpy())

#     acc = num_corrects / num_total
#     auc = roc_auc_score(labels, outs)
#     loss = np.mean(valid_loss)

#     return loss, acc, auc

In [60]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = SAKTModel(n_skill, max_seq=MAX_SEQ, embed_dim=EMBED_DIM, dropout_rate=DROPOUT_RATE)
# optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# criterion = nn.BCEWithLogitsLoss()
# scheduler = torch.optim.lr_scheduler.OneCycleLR(
#     optimizer, max_lr=MAX_LEARNING_RATE, steps_per_epoch=len(train_dataloader), epochs=EPOCHS
# )

# model.to(device)
# criterion.to(device)
# model

In [61]:
# def train_sakt(n_folds=0):
#     best_auc = 0
#     max_steps = 3
#     step = 0
#     for epoch in tqdm(range(EPOCHS)):
#         loss, acc, auc = train_fn(model, train_dataloader, optimizer, scheduler, criterion, device)
#         print("epoch - {}/{} train: - {:.3f} acc - {:.3f} auc - {:.3f}".format(epoch+1, EPOCHS, loss, acc, auc))
#         loss, acc, auc = valid_fn(model, valid_dataloader, criterion, device)
#         print("epoch - {}/{} valid: - {:.3f} acc - {:.3f} auc - {:.3f}".format(epoch+1, EPOCHS, loss, acc, auc))
        
#         if auc > best_auc:
#             best_auc = auc
#             step = 0
#             torch.save(model.state_dict(), f"sakt_model{n_folds}.pt")
#         else:
#             step += 1
#             if step >= max_steps:
#                 break
                
#     return best_auc

## モデルTraining

In [62]:
print(tr.shape, val.shape)
print('')
for cols in [feature_cols,cat_feature_cols, target_col, num_feature_cols]:
    print(cols)
    print(len(cols))
    print('')

(17755881, 52) (2244119, 52)

['content_id', 'timestamp', 'task_container_id', 'prior_question_elapsed_time', 'prior_question_had_explanation', 'bundle_id', 'part', 'content_cnt', 'content_inccnt', 'content_corrate', 'content_avg_time', 'content_expl_avg', 'new_content_corcnt', 'new_content_corrate', 'same_bundle_cnt', 'bundle_corrate', 'tags1', 'tags2', 'new_w2v', 'elapsed_time_u_avg', 'explanation_u_avg', 'timestamp_u_correct_recency', 'timestamp_u_incorrect_recency', 'timestamp_u_diff_0_1', 'timestamp_u_diff_1_2', 'timestamp_u_diff_2_3', 'task_container_id_back', 'user_cnt', 'user_corcnt', 'user_inccnt', 'user_corrate', 'user_diff_sum_corrate', 'user_diff_avg_corrate', 'user_roll10_corrate', 'user_roll30_corrate', 'user_diff_sum_time', 'user_diff_avg_time', 'user_past_content_cnt', 'user_part_cnt', 'user_part_corcnt', 'user_part_inccnt', 'user_part_corrate', 'user_part_diff_sum_corrate', 'user_part_diff_avg_corrate', 'user_part_roll5_corrate', 'user_part_roll15_corrate', 'user_w2v_c

### GBDT

In [63]:
start = time.time()
print(datetime.datetime.now())

2021-01-06 17:31:40.506847


In [ ]:
#xgb
# xgb_auc, xgb_model = train_xgb(tr, val, feature_cols, target_col, num_iter=10000)
#lgbm
# lgb_auc, lgb_model = train_lgbm(tr, val, feature_cols, cat_feature_cols, target_col, num_iter=10000)
#ctb
ctb_auc, ctb_model = train_ctb(tr, val, feature_cols, target_col, num_iter=100000)

0:	test: 0.7554230	best: 0.7554230 (0)	total: 1.89s	remaining: 2d 4h 37m 55s
100:	test: 0.7720553	best: 0.7720553 (100)	total: 2m 32s	remaining: 1d 17h 47m 52s


In [ ]:
finish = time.time()
print(datetime.datetime.now())
# print(finish - start)

2021-01-06 21:39:18.901822


In [ ]:
print(
#     xgb_auc,
#     lgb_auc,
    ctb_auc
)

0.7998176581697033


In [ ]:
#save models
# with open(f'xgb_model.pkl', mode='wb') as fp:
#     pickle.dump(xgb_model, fp)
# with open(f'lgb_gcp_2.pkl', mode='wb') as fp:
#     pickle.dump(lgb_model, fp, protocol=4)
#catboostだけ特殊
ctb_model.save_model('ctb_model_20mil_100th.cbm')

In [70]:
#load
with open(f'ctb_model_20mil_100th.pkl', mode='wb') as fp:
    pickle.dump(ctb_model, fp)
    
with open('ctb_model_20mil_100th.pkl', mode='rb') as fp:
    ctb_model_restore = pickle.load(fp)
#catboost
ctb_model_restore = ctb.CatBoost()
ctb_model_restore.load_model('ctb_model_20mil_100th.cbm')

In [71]:
# print(xgb_model.best_iteration == xgb_model_restore.best_iteration)
# print(lgb_model.best_iteration == lgb_model_restore.best_iteration)
print(ctb_model == ctb_model_restore)

True


### SAKT

In [ ]:
sakt_auc = train_sakt(n_folds=0)

In [ ]:
# test_dataset = TestDataset(test, num_features, cat_features)
#     test_loader = DataLoader(test_dataset, batch_size=cfg.batch_size, shuffle=False, 
#                              num_workers=4, pin_memory=True)
# model = TabularNN(cfg)
# model.load_state_dict(torch.load(f"fold{fold_num}_seed{seed}.pth"))
# model.to(device)
# predictions = inference_fn(test_loader, model, device)

# # del
# torch.cuda.empty_cache()